In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from IPython.display import display


In [2]:
df = pd.read_csv('train_data.csv')
dft = pd.read_csv('test.csv')

In [3]:
# Drop the columns not used in the model
df = df.drop(columns=['country_code','fc_codes'])
df

,ofd_date,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline
0,2021-06-30,D33,14594,14568,782,896,615,767,35,2,4
1,2021-06-30,D37,12736,13111,655,823,211,29,17,2,1
2,2021-06-30,D34,14562,15651,1028,1910,225,35,47,3,1
3,2021-06-30,D45,11165,11467,514,769,56,39,29,0,1
4,2021-06-30,D50,10006,10423,399,842,52,60,65,1,1
...,...,...,...,...,...,...,...,...,...,...,...
11304,2021-02-01,D60,16025,15768,487,383,62,69,195,15,1
11305,2021-02-01,D54,18986,18861,318,446,83,100,107,248,1
11306,2021-02-01,D69,12479,12345,163,196,64,64,62,136,0
11307,2021-02-01,D59,16828,16504,248,333,83,135,112,256,1


In [4]:
# create Target variable

df = df.assign(Target = df.Earlies_Exp - df.MNR_SNR_Exp)

In [5]:
df = df.assign(PK = df.ofd_date + '_' + df.station_code )
df["PK"] = df['ofd_date'] + '_' + df['station_code']
df.head()

,ofd_date,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline,Target,PK
0,2021-06-30,D33,14594,14568,782,896,615,767,35,2,4,167,2021-06-30_D33
1,2021-06-30,D37,12736,13111,655,823,211,29,17,2,1,444,2021-06-30_D37
2,2021-06-30,D34,14562,15651,1028,1910,225,35,47,3,1,803,2021-06-30_D34
3,2021-06-30,D45,11165,11467,514,769,56,39,29,0,1,458,2021-06-30_D45
4,2021-06-30,D50,10006,10423,399,842,52,60,65,1,1,347,2021-06-30_D50


In [6]:
# Drop the columns not used in the model
df = df.drop(columns=['Earlies_Exp','MNR_SNR_Exp', 'ofd_date'])

In [7]:
# differentiate numerical features (minus the target) and categorical features

categorical_features = df.select_dtypes(include = ["object"]).columns
numerical_features = df.select_dtypes(exclude = ["object"]).columns

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
df_num = df[numerical_features]
df_cat = df[categorical_features]

Numerical features : 8
Categorical features : 2


In [8]:
# Drop the columns not used in the model
df = df.drop(columns=['station_code'])

df.head()

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,Target,PK
0,14594,14568,896,767,35,2,4,167,2021-06-30_D33
1,12736,13111,823,29,17,2,1,444,2021-06-30_D37
2,14562,15651,1910,35,47,3,1,803,2021-06-30_D34
3,11165,11467,769,39,29,0,1,458,2021-06-30_D45
4,10006,10423,842,60,65,1,1,347,2021-06-30_D50


In [9]:
# check features

target = 'Target'
features = [i for i in df.columns if i not in [target]]

print('\n\033[1mInference:\033[0m The Dataset consists of {} columns & {} samples.'.format(df.shape[1], df.shape[0]))


Inference: The Dataset consists of 9 columns & 11309 samples.


In [10]:
# removal of outlier:

df1 = df
df3 = df

#features1 = [i for i in features if i not in ['CHAS','RAD']]
features1 = numerical_features

for i in features1:
    Q1 = df1[i].quantile(0.0005)
    Q3 = df1[i].quantile(0.95)
    IQR = Q3 - Q1
    df1 = df1[df1[i] <= (Q3+(1.5*IQR))]
    df1 = df1[df1[i] >= (Q1-(1.5*IQR))]
    df1 = df1.reset_index(drop=True)
display(df1.head())
print('\n\033[1mInference:\033[0m\nBefore removal of outliers, The dataset had {} samples.'.format(df3.shape[0]))
print('After removal of outliers, The dataset now has {} samples.'.format(df1.shape[0]))

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,Target,PK
0,14594,14568,896,767,35,2,4,167,2021-06-30_D33
1,12736,13111,823,29,17,2,1,444,2021-06-30_D37
2,14562,15651,1910,35,47,3,1,803,2021-06-30_D34
3,11165,11467,769,39,29,0,1,458,2021-06-30_D45
4,10006,10423,842,60,65,1,1,347,2021-06-30_D50



Inference:
Before removal of outliers, The dataset had 11309 samples.
After removal of outliers, The dataset now has 10800 samples.


In [11]:
# removal of outlier:

df1 = df
df3 = df

#features1 = [i for i in features if i not in ['CHAS','RAD']]
features1 = numerical_features

for i in features1:
    Q1 = df1[i].quantile(0.05)
    Q3 = df1[i].quantile(0.95)
    IQR = Q3 - Q1
    df1 = df1[df1[i] <= (Q3+(1.5*IQR))]
    df1 = df1[df1[i] >= (Q1-(1.5*IQR))]
    df1 = df1.reset_index(drop=True)
display(df1.head())
print('\n\033[1mInference:\033[0m\nBefore removal of outliers, The dataset had {} samples.'.format(df3.shape[0]))
print('After removal of outliers, The dataset now has {} samples.'.format(df1.shape[0]))

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,Target,PK
0,14594,14568,896,767,35,2,4,167,2021-06-30_D33
1,12736,13111,823,29,17,2,1,444,2021-06-30_D37
2,14562,15651,1910,35,47,3,1,803,2021-06-30_D34
3,11165,11467,769,39,29,0,1,458,2021-06-30_D45
4,10006,10423,842,60,65,1,1,347,2021-06-30_D50



Inference:
Before removal of outliers, The dataset had 11309 samples.
After removal of outliers, The dataset now has 10759 samples.


In [12]:
df = df1
df.shape

(10759, 9)

In [13]:
d = pd.get_dummies(df, prefix = ["theday"], columns = ["PK"])
d.head()

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,Target,theday_2021-02-01_D1,theday_2021-02-01_D10,...,theday_2021-06-30_D71,theday_2021-06-30_D72,theday_2021-06-30_D73,theday_2021-06-30_D74,theday_2021-06-30_D75,theday_2021-06-30_D76,theday_2021-06-30_D77,theday_2021-06-30_D79,theday_2021-06-30_D8,theday_2021-06-30_D9
0,14594,14568,896,767,35,2,4,167,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12736,13111,823,29,17,2,1,444,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14562,15651,1910,35,47,3,1,803,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11165,11467,769,39,29,0,1,458,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10006,10423,842,60,65,1,1,347,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# dependent and independent variables 
a = d.drop(columns=["Target"])
b = d["Target"]

In [15]:
d.shape

(10759, 10767)

In [16]:
dft.head()

,Unnamed: 0,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline
0,0,2021-07-31,C,"F6, F8, F14, F17",D33,10231,10957,347,43,42,2,629
1,1,2021-07-31,C,"F6, F8, F9, F14, F17, F18",D37,10113,10261,285,50,25,0,165
2,2,2021-07-31,C,"F1, F4, F6, F7, F13, F15, F16",D34,11633,11789,300,24,30,0,202
3,3,2021-07-31,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,8633,8482,180,713,24,1,276
4,4,2021-07-31,C,"F6, F8, F13, F14, F17",D50,6972,7222,116,24,16,0,130


In [17]:
# Drop the columns not used in the model
dft = dft.drop(columns=['country_code','fc_codes'])
dft

,Unnamed: 0,ofd_date,station_code,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline
0,0,2021-07-31,D33,10231,10957,347,43,42,2,629
1,1,2021-07-31,D37,10113,10261,285,50,25,0,165
2,2,2021-07-31,D34,11633,11789,300,24,30,0,202
3,3,2021-07-31,D45,8633,8482,180,713,24,1,276
4,4,2021-07-31,D50,6972,7222,116,24,16,0,130
...,...,...,...,...,...,...,...,...,...,...
2384,2384,2021-07-01,D60,11782,14462,113,34,88,0,0
2385,2385,2021-07-01,D54,16615,16643,222,37,95,1,0
2386,2386,2021-07-01,D69,10733,10801,54,26,64,1,0
2387,2387,2021-07-01,D59,14633,14650,91,51,80,1,0


In [18]:
print(dft.shape)
print(d.shape)

(2389, 10)
(10759, 10767)


In [19]:
dft["Target"] = " "
dft.head()

print(dft.shape)
print(d.shape)

(2389, 11)
(10759, 10767)


In [20]:
dft = dft.assign(PK = dft.ofd_date + '_' + dft.station_code )
dft["PK"] = dft['ofd_date'] + '_' + dft['station_code']
dft.head()

print(dft.shape)
print(d.shape)

(2389, 12)
(10759, 10767)


In [21]:
# Drop the columns not used in the model
dft = dft.drop(columns=['ofd_date', 'station_code'])
print(dft.shape)
print(d.shape)

(2389, 10)
(10759, 10767)


In [22]:
dd = pd.get_dummies(dft, prefix = ["theday"], columns = ["PK"])
print(dft.shape)
print(d.shape)

(2389, 10)
(10759, 10767)


In [23]:
dd = dd.reindex(labels=d.columns,axis=1)
print(dd.shape)
print(d.shape)

(2389, 10767)
(10759, 10767)


In [24]:
dd = dd.fillna(0)

print(dd)

        OFD   Slam  Earlies_Rec  Rollover  Returns  R_Sideline  Sideline  \
0     10231  10957          347        43       42           2       629   
1     10113  10261          285        50       25           0       165   
2     11633  11789          300        24       30           0       202   
3      8633   8482          180       713       24           1       276   
4      6972   7222          116        24       16           0       130   
...     ...    ...          ...       ...      ...         ...       ...   
2384  11782  14462          113        34       88           0         0   
2385  16615  16643          222        37       95           1         0   
2386  10733  10801           54        26       64           1         0   
2387  14633  14650           91        51       80           1         0   
2388  15562  15502          173        44       92          10         1   

     Target  theday_2021-02-01_D1  theday_2021-02-01_D10  ...  \
0                     

In [25]:
dd.head()

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,Target,theday_2021-02-01_D1,theday_2021-02-01_D10,...,theday_2021-06-30_D71,theday_2021-06-30_D72,theday_2021-06-30_D73,theday_2021-06-30_D74,theday_2021-06-30_D75,theday_2021-06-30_D76,theday_2021-06-30_D77,theday_2021-06-30_D79,theday_2021-06-30_D8,theday_2021-06-30_D9
0,10231,10957,347,43,42,2,629,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10113,10261,285,50,25,0,165,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11633,11789,300,24,30,0,202,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8633,8482,180,713,24,1,276,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6972,7222,116,24,16,0,130,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
X = d.drop(columns=["Target"])
Y = d["Target"]
print(X.shape)
print(Y.shape)

(10759, 10766)
(10759,)


In [27]:
x = dd.drop(columns=["Target"])
y = dd["Target"]
print(x.shape)
print(y.shape)

(2389, 10766)
(2389,)


In [28]:
q = X.values
w = Y.values
e = x.values
f = y.values


In [29]:

from sklearn import linear_model

regressor = linear_model.Lasso(alpha=0.1)

regressor.fit(q, w)


/Users/andreslopez/Desktop/dataton copy/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.662e+08, tolerance: 3.388e+05
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [30]:
print(f.shape)

(2389,)


In [31]:
print(e)

[[10231. 10957.   347. ...     0.     0.     0.]
 [10113. 10261.   285. ...     0.     0.     0.]
 [11633. 11789.   300. ...     0.     0.     0.]
 ...
 [10733. 10801.    54. ...     0.     0.     0.]
 [14633. 14650.    91. ...     0.     0.     0.]
 [15562. 15502.   173. ...     0.     0.     0.]]


In [32]:
y_pred = regressor.predict(x)

/Users/andreslopez/Desktop/dataton copy/venv/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [33]:
print(y_pred)

[290.9227359  261.65281043 317.9028981  ... -36.09527747  28.62463946
 146.67510457]


In [34]:
y_pred.shape

(2389,)

In [35]:
print(y_pred[: 30])

[ 290.9227359   261.65281043  317.9028981   147.01147653   17.33423659
  173.68061464  132.46415108  -28.55593946  127.7088238    -1.07959517
  -23.50087196   62.44748657   66.84572939   87.99006551   53.35096309
 -237.00367533   32.02805604  223.38869814  100.34123415   28.55064083
  197.43809378   97.65986181  239.63382993   65.30732836  -59.81043384
  -65.26863074  -41.90248075  -38.1001881   264.0310425   209.58238121]


In [36]:
lazy = dft['PK'].values
print(lazy)

['2021-07-31_D33' '2021-07-31_D37' '2021-07-31_D34' ... '2021-07-01_D69'
 '2021-07-01_D59' '2021-07-01_D57']


In [37]:
df3 = pd.DataFrame(y_pred)

df4 = pd.DataFrame(lazy)


In [38]:
df3['PK'] = df4

In [39]:
df3.head()

,0,PK
0,290.922736,2021-07-31_D33
1,261.652810,2021-07-31_D37
2,317.902898,2021-07-31_D34
3,147.011477,2021-07-31_D45
4,17.334237,2021-07-31_D50


In [40]:
df5 = pd.DataFrame(y_pred)
df4["Expected"] = df5
df4.head()

,0,Expected
0,2021-07-31_D33,290.922736
1,2021-07-31_D37,261.652810
2,2021-07-31_D34,317.902898
3,2021-07-31_D45,147.011477
4,2021-07-31_D50,17.334237


In [41]:
df99 = df4

df76= df99.rename(columns={0: 'Id'})



df76.head()

,Id,Expected
0,2021-07-31_D33,290.922736
1,2021-07-31_D37,261.652810
2,2021-07-31_D34,317.902898
3,2021-07-31_D45,147.011477
4,2021-07-31_D50,17.334237


In [42]:
csv_data = df76.to_csv(columns=['Id', 'Expected'], index=False)
print(csv_data)

Id,Expected
2021-07-31_D33,290.9227359022063
2021-07-31_D37,261.6528104276603
2021-07-31_D34,317.90289809566406
2021-07-31_D45,147.0114765316901
2021-07-31_D50,17.334236590073743
2021-07-31_D32,173.68061463939404
2021-07-31_D49,132.4641510831669
2021-07-31_D48,-28.55593945736001
2021-07-31_D42,127.70882380428768
2021-07-31_D38,-1.0795951670965138
2021-07-31_D43,-23.50087195751179
2021-07-31_D39,62.44748656780325
2021-07-31_D36,66.84572939006594
2021-07-31_D35,87.99006551092428
2021-07-31_D40,53.35096309246066
2021-07-31_D47,-237.0036753343955
2021-07-31_D41,32.02805604110659
2021-07-31_D46,223.3886981404089
2021-07-31_D44,100.34123414907555
2021-07-31_D31,28.55064083244895
2021-07-31_D4,197.43809377871142
2021-07-31_D9,97.65986180765123
2021-07-31_D1,239.63382993232966
2021-07-31_D10,65.3073283640671
2021-07-31_D5,-59.81043383916008
2021-07-31_D7,-65.26863074142567
2021-07-31_D3,-41.9024807499476
2021-07-31_D6,-38.100188098153936
2021-07-31_D8,264.03104249930226
2021-07-31_D2,209.58238